In [ ]:
% matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import os
import sys
from IPython.display import Image


# Add modules path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Translator import Translator
from DecisionTree import DecisionTree
import preprocessor as pp


# Add modules path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Define language and translator functions
language = "es"
translator = Translator(language)
to_lang = translator.translate_to_language
to_col = translator.translate_to_column

# Load data
raw_data = pd.read_csv("sample.txt", header=0, skiprows=1, delimiter="\t", index_col=0, usecols=list(range(0, 9)),
                       parse_dates=to_lang(["Datetime"]), decimal=",",
                       date_parser=lambda x: pd.to_datetime(x, format="%Y/%m/%d %H:%M"))
# Translate column names
raw_data.columns = (to_col(raw_data.columns))

# Divide in blocks, extend dataset and clean data
block_data = pp.define_blocks(raw_data)
cleaned_block_data = pp.clean_processed_data(block_data)
extended_data = pp.extend_data(cleaned_block_data)
cleaned_extended_data = pp.clean_extended_data(extended_data)

# Create decision trees
[data, labels] = pp.prepare_to_decision_trees (cleaned_extended_data)
hyper_dt = DecisionTree(data, labels["Hyperglycemia_Diagnosis"])
hypo_dt = DecisionTree(data, labels["Hypoglycemia_Diagnosis"])

In [17]:
hyper_dt.cross_val_score(scoring='roc_auc')

0.83511660671040766

In [6]:
hyper_dt.cross_val_score(scoring='accuracy')

0.89059383893592459

In [16]:
hyper_dt.label[hyper_dt.label == 0].shape[0]/hyper_dt.data.shape[0]

0.8905887807139545

In [15]:
hyper_dt.cross_val_score(scoring='recall')

0.0

In [11]:
hyper_dt.cross_val_score(scoring='average_precision')

0.59920502646179308

Recall is 0 because the model is not able to classiffy any example to a positive label
(In this case, hyperglucemya). That makes it a bad classifier but still may provide interesting
patterns if the ratio of positive and negative samples of one leaf of the tree is balanced even if the
prediction is negative.